# High Cardinality Encoding

## Datasets with high cardinality = too many variables

[Pandas for One-Hot Encoding Data Preventing High Cardinality](https://towardsdatascience.com/pandas-for-one-hot-encoding-data-preventing-high-cardinality-16148a0ca399)

[OHE_Dimensionality.ipynb](https://github.com/gurezende/Studying/blob/master/Python/Pandas/OHE_Dimensionality.ipynb)

[Book2.csv](https://github.com/gurezende/Studying/blob/master/Python/Pandas/Book2.csv)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Book2.csv')
# df.head(5)
df.sample(5)

,city,product_dsc,qty,total
289,Paris,FRC,7,105
270,Paris,FRC,2,30
230,Paris,FRC,9,135
2,NYC,ABC,4,48
152,Rome,ITC,5,50


In [3]:
# Count number of distinct cities
df.city.nunique()

24

In [4]:
# Count number of unique products
df.product_dsc.nunique()

21

Notice the high number of different cities and products. If we use OHE for this dataset, we'll get a very large dataset.

In [5]:
# One Hot Encoding
fluff = pd.get_dummies(df, drop_first=True).head()

# drop-first removed: city_Austin, product_dsc_AAA, city2_NYC, and product2_ABC.
fluff

,qty,total,city_Baltimore,city_Boston,city_El Paso,city_Florence,city_Le Mans,city_Lille,city_Los Angeles,city_Lyon,...,product_dsc_DGF,product_dsc_FFF,product_dsc_FRC,product_dsc_GDS,product_dsc_ITC,product_dsc_SFS,product_dsc_SSD,product_dsc_VVV,product_dsc_WES,product_dsc_WW
0,1,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,84,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,48,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,36,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Even using the argument `drop_first=True` to discard the first category of each variable, we still get 45 variables.

In [6]:
# Counting by city
df.city.value_counts()
# df.city.value_counts(normalize=True)  # if want to see %

Rome             145
Paris            124
NYC               47
Lyon               3
Naples             2
Boston             2
Milan              1
Le Mans            1
Nantes             1
Nice               1
Lille              1
Raleigh            1
Florence           1
Venice             1
Richmond           1
Miami              1
Austin             1
El Paso            1
Tulsa              1
San Francisco      1
Los Angeles        1
Baltimore          1
Pittsburgh         1
St Dennis          1
Name: city, dtype: int64

In [7]:
# Counting by product
df.product_dsc.value_counts()

ITC    148
FRC    126
ABC     47
SSD      2
WW       2
CC       1
GDS      1
FFF      1
DDS      1
DDE      1
DD       1
CDE      1
WES      1
VVV      1
BBB      1
AAA      1
DDF      1
DDD      1
DGF      1
DFS      1
SFS      1
Name: product_dsc, dtype: int64

We can see that there are only 3 cities and 3 products that make approx. 90% of the data. So, in cases like this, the simple solution proposed here is to use Pandas to reduce the number of categories, keeping only those top values as is and gather everything else under an "other" bucket.

In [8]:
# LIST TOP 3
city_top3 = df.groupby('city').city.count().nlargest(3).index
# 'Rome', 'Paris', 'NYC'

prod_top3 = df.groupby('product_dsc').product_dsc.count().nlargest(3).index
# 'ITC', 'FRC', 'ABC'

In [9]:
# Keep top 3 as is and the rest as "Other"
df['city2'] = df.city.where(df.city.isin(city_top3), other='Other')
df['product2'] = df.product_dsc.where(df.product_dsc.isin(prod_top3), other='Other')

df['city2']

0        NYC
1        NYC
2        NYC
3      Other
4      Other
       ...  
336    Paris
337    Paris
338    Paris
339    Paris
340    Paris
Name: city2, Length: 341, dtype: object

In [10]:
df['product2']

0      ABC
1      ABC
2      ABC
3      ABC
4      ABC
      ... 
336    FRC
337    FRC
338    FRC
339    FRC
340    FRC
Name: product2, Length: 341, dtype: object

Let's look at a sample of the resulting data so far.

In [11]:
df.sample(6)

,city,product_dsc,qty,total,city2,product2
293,Paris,FRC,10,150,Paris,FRC
103,Rome,ITC,1,10,Rome,ITC
57,NYC,WW,6,42,NYC,Other
149,Rome,ITC,5,50,Rome,ITC
280,Paris,FRC,3,45,Paris,FRC
30,NYC,ABC,5,60,NYC,ABC


Next, we can just drop the old variables `city` and `product_dsc` and we're ready to use OHE again.

In [12]:
# Drop old city and product
df2 = df.drop(['city', 'product_dsc'], axis=1)
df2.head()

,qty,total,city2,product2
0,1,12,NYC,ABC
1,7,84,NYC,ABC
2,4,48,NYC,ABC
3,3,36,Other,ABC
4,2,24,Other,ABC


In [13]:
aFrame = pd.get_dummies(df2)

aFrame.shape  # yes, you can get shape from a dataframe :)

(341, 10)

In [14]:
aFrame.head()

,qty,total,city2_NYC,city2_Other,city2_Paris,city2_Rome,product2_ABC,product2_FRC,product2_ITC,product2_Other
0,1,12,1,0,0,0,1,0,0,0
1,7,84,1,0,0,0,1,0,0,0
2,4,48,1,0,0,0,1,0,0,0
3,3,36,0,1,0,0,1,0,0,0
4,2,24,0,1,0,0,1,0,0,0


Notice that we reduced the number of variables after OHE from 45 to only 10 columns!

## Summary

* One Hot Encoding is useful to transform categorical data into numbers.
* Using OHE in a dataset with too many variable will create a wide dataset.
* Too wide data can suffer with "the Curse of Dimensionality", putting the performance of the model in jeopardy.
* A simple solution can be using Pandas `.groupby()`, `.nlargest()` and `.where()` to reduce the number of categories prior to OHE.